# Step 1: import 必要套件

In [1]:
from main.model.squeezeDet import SqueezeDet
from main.model.dataGenerator import generator_from_data_path
from main.config.create_config import load_dict
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import optimizers
import pickle
import os
import gc

# Step 2: 基本設定 & 載入檔案
* img_train.txt
* gt_train.txt
* squeeze.config

In [2]:
img_file = "img_train.txt"
gt_file  = "gt_train.txt"
log_dir_name = './log'
EPOCHS = 100
STEPS = None
OPTIMIZER = "default"
REDUCELRONPLATEAU = True
VERBOSE = True
CONFIG = "squeeze.config"

tb_dir = log_dir_name +"/tensorboard"

if "log" not in os.listdir():
    os.makedirs(tb_dir)

In [3]:
# 目前有 img_names 與 gt_names 還沒有 sorted 的情況
with open(img_file) as imgs:
    img_names = imgs.read().splitlines()
imgs.close()

In [4]:
with open(gt_file) as gts:
    gt_names = gts.read().splitlines()
gts.close()

In [5]:
cfg = load_dict(CONFIG)

cfg.img_file = img_file
cfg.gt_file = gt_file
cfg.images = img_names
cfg.gts = gt_names
#cfg.init_file = init_file
cfg.EPOCHS = EPOCHS
cfg.OPTIMIZER = OPTIMIZER
#cfg.CUDA_VISIBLE_DEVICES = CUDA_VISIBLE_DEVICES
#cfg.GPUS = GPUS
cfg.REDUCELRONPLATEAU = REDUCELRONPLATEAU

In [6]:
train_generator = generator_from_data_path(img_names, gt_names, config=cfg)

In [7]:
imgs , labs = next(train_generator)

In [8]:
# compute number of batches per epoch
nbatches_train, mod = divmod(len(img_names), cfg.BATCH_SIZE)
print(f"Number of images: {len(img_names)}")
print(f"Number of epochs: {EPOCHS}")
print(f"Number of batches: {nbatches_train}")
print(f"Batch size: {cfg.BATCH_SIZE}")

Number of images: 5984
Number of epochs: 100
Number of batches: 1496
Batch size: 4


In [9]:
# instantiate model
squeeze = SqueezeDet(cfg)

In [10]:
# 決定 Optimizer，執行完這格，會是 SGD
if OPTIMIZER == "adam":
    opt = optimizers.Adam(lr=0.001,  clipnorm=cfg.MAX_GRAD_NORM)
    cfg.LR= 0.001
if OPTIMIZER == "rmsprop":
    opt = optimizers.RMSprop(lr=0.001,  clipnorm=cfg.MAX_GRAD_NORM)
    cfg.LR= 0.001

if OPTIMIZER == "adagrad":
    opt = optimizers.Adagrad(lr=1.0,  clipnorm=cfg.MAX_GRAD_NORM)
    cfg.LR = 1

#use default is nothing is given
else:
    # create sgd with momentum and gradient clipping
    opt = optimizers.SGD(lr=cfg.LEARNING_RATE, decay=0, momentum=cfg.MOMENTUM,
                         nesterov=False, clipnorm=cfg.MAX_GRAD_NORM)

    cfg.LR = cfg.LEARNING_RATE 


    print("Learning rate: {}".format(cfg.LEARNING_RATE))

Learning rate: 0.01


In [11]:
#save config file to log dir
with open(log_dir_name  +'/config.pkl', 'wb') as f:
    pickle.dump(cfg, f, pickle.HIGHEST_PROTOCOL)

In [12]:
#callbacks
cb = []

#add tensorboard callback
tbCallBack = TensorBoard(log_dir=tb_dir, histogram_freq=0, write_graph=True, write_images=True)
cb.append(tbCallBack)

if REDUCELRONPLATEAU:

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,verbose=1, patience=5, min_lr=0.0)

    cb.append(reduce_lr)

if VERBOSE:
    print(squeeze.model.summary())
    
    # will (None, 16848, 12) work in this case? I doubt about it.

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 384, 1248, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 192, 624, 64) 1792        input[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 96, 312, 64)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
fire2/squeeze1x1 (Conv2D)       (None, 96, 312, 16)  1040        pool1[0][0]                      
_______________________________________________________________________________________

In [13]:

# compile model
squeeze.model.compile(optimizer=opt, loss=[squeeze.loss], metrics=[squeeze.loss_without_regularization, squeeze.bbox_loss, squeeze.class_loss, squeeze.conf_loss])

In [ ]:
# To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

tf.autograph.experimental.do_not_convert(
    squeeze.model.fit_generator(train_generator, 
                                epochs=EPOCHS, 
                                steps_per_epoch=nbatches_train, 
                                callbacks=cb
    )
)

Epoch 1/100
1496/1496 [==============================] - 1354s 905ms/step - loss: 4.1253 - loss_without_regularization: 4.0256 - bbox_loss: 2.2352 - class_loss: 0.6680 - conf_loss: 1.1225
Epoch 2/100
1496/1496 [==============================] - 1343s 898ms/step - loss: 3.7942 - loss_without_regularization: 3.7260 - bbox_loss: 2.1698 - class_loss: 0.6201 - conf_loss: 0.9361
Epoch 3/100
1496/1496 [==============================] - 1403s 938ms/step - loss: 3.7332 - loss_without_regularization: 3.6796 - bbox_loss: 2.1345 - class_loss: 0.6230 - conf_loss: 0.9221
Epoch 4/100
1027/1496 [===================>..........] - ETA: 7:13 - loss: 3.6442 - loss_without_regularization: 3.6018 - bbox_loss: 2.0877 - class_loss: 0.6193 - conf_loss: 0.8949